In [1]:
import pandas as pd
import requests

C:\Users\amarl\AppData\Local\Temp\ipykernel_23364\1058106858.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
resp = requests.get('https://barttorvik.com/getgamestats.php?sIndex=7&year=2023&tvalue=All&cvalue=All&opcvalue=All&ovalue=All&minwin=All&mindate=11/06/2023&maxdate=11/07/2023&typev=All&venvalue=All&minadjo=0&minadjd=200&mintempo=0&minppp=0&minefg=0&mintov=200&minreb=0&minftr=0&minpppd=200&minefgd=200&mintovd=0&minrebd=200&minftrd=200&mings=0&mingscript=-100&maxx=100&coach=All&opcoach=All&adjoSelect=min&adjdSelect=max&tempoSelect=min&pppSelect=min&efgSelect=min&tovSelect=max&rebSelect=min&ftrSelect=min&pppdSelect=max&efgdSelect=max&tovdSelect=min&rebdSelect=max&ftrdSelect=max&gscriptSelect=min&sortToggle=1')

In [3]:
resp.json()[0]

['2/4/23',
 1,
 'Abilene Christian',
 'WAC',
 'Cal Baptist',
 'A',
 'W, 87-71',
 131.5,
 115.4,
 133,
 62.5,
 16.8,
 34.6,
 48.1,
 108.6,
 52.8,
 19.9,
 36.4,
 43.4,
 81.8,
 'WAC',
 1,
 2023,
 65.4,
 'Abilene ChristianCal Baptist2-4',
 'Brette Tanner',
 'Rick Croy',
 0.566046,
 0.593176,
 '["2/4/2023", 200, "Abilene Christian", "Cal Baptist", 27, 52, 11, 20, 22, 25, 9, 21, 30, 15, 7, 1, 11, 20, 87, 25, 53, 6, 25, 15, 23, 12, 17, 29, 16, 4, 3, 13, 16, 71, 65.4, "", "Abilene Christian", "Cal Baptist"]',
 0]

In [4]:
header = ['Date', 'Game_Type', 'Team', 'Conference', 'Opponent', 'Location', 'Result', 'Adj_Offense', 'Adj_Defense', 'Offensive_Efficiency',  'Effective_FG_Percent', 'Turnover_Percent', 'Offensive_Rebound_Percent', 'Free_Throw_Rate', 'Opponent_Offensive_Efficiency', 'Opponent_Effective_FG_Percent', 'Opponent_Turnover_Percent', 'Opponent_Offensive_Rebound_Percent', 'Opponent_Free_Throw_Rate', 'UNK2', 'Opponent_Conference', 'UNK2', 'Season', 'UNK3', 'UNK4', 'UNK5', 'UNK6', 'UNK7', 'UNK8', 'UNK9', 'UNK10']

In [5]:
len(header) # same length, nice


31

In [6]:
relevant_fields = [field for field in header if not field.startswith('UNK')]

In [7]:
df = pd.DataFrame(resp.json(), columns=header)
df = df[relevant_fields]

In [70]:
import numpy
print("create table game_stats (")
for col in relevant_fields:
    if type(df[col][0]) == str:
        print(f"{col} varchar,")
    elif type(df[col][0]) == numpy.int64:
        print(f"{col} int,")
    elif type(df[col][0]) == numpy.float64:
        print(f"{col} numeric(10,10),")
    else:
        print(f"unknown type {type(df[col][0])} for {col}")

create table game_stats (
Date varchar,
Game_Type int,
Team varchar,
Conference varchar,
Opponent varchar,
Location varchar,
Result varchar,
Adj_Offense numeric(10,10),
Adj_Defense numeric(10,10),
Offensive_Efficiency numeric(10,10),
Effective_FG_Percent numeric(10,10),
Turnover_Percent numeric(10,10),
Offensive_Rebound_Percent numeric(10,10),
Free_Throw_Rate numeric(10,10),
Opponent_Offensive_Efficiency numeric(10,10),
Opponent_Effective_FG_Percent numeric(10,10),
Opponent_Turnover_Percent numeric(10,10),
Opponent_Offensive_Rebound_Percent numeric(10,10),
Opponent_Free_Throw_Rate numeric(10,10),
Opponent_Conference varchar,
Season int,


In [8]:
import os
import sqlalchemy
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [14]:
conn.commit()

In [16]:
conn.execute(sqlalchemy.text("delete from sports_data.game_stats"))
for i, row in df.iterrows():
    insert_statement = f"""insert into sports_data.game_stats values ('{"','".join([str(x).replace("'","''") for x in row])}')"""
    conn.execute(sqlalchemy.text(insert_statement))
conn.commit()

In [17]:
for year in ["2024", "2022", "2021", "2020", "2019", "2018", "2017", "2016", "2015"]:
    resp = requests.get(f'https://barttorvik.com/getgamestats.php?sIndex=7&year={year}&tvalue=All&cvalue=All&opcvalue=All&ovalue=All&minwin=All&mindate=11/06/2023&maxdate=11/07/2023&typev=All&venvalue=All&minadjo=0&minadjd=200&mintempo=0&minppp=0&minefg=0&mintov=200&minreb=0&minftr=0&minpppd=200&minefgd=200&mintovd=0&minrebd=200&minftrd=200&mings=0&mingscript=-100&maxx=100&coach=All&opcoach=All&adjoSelect=min&adjdSelect=max&tempoSelect=min&pppSelect=min&efgSelect=min&tovSelect=max&rebSelect=min&ftrSelect=min&pppdSelect=max&efgdSelect=max&tovdSelect=min&rebdSelect=max&ftrdSelect=max&gscriptSelect=min&sortToggle=1')
    df = pd.DataFrame(resp.json(), columns=header)
    df = df[relevant_fields]
    for i, row in df.iterrows():
        insert_statement = f"""insert into sports_data.game_stats values ('{"','".join([str(x).replace("'","''") for x in row])}')"""
        conn.execute(sqlalchemy.text(insert_statement))
        
    conn.commit()

In [86]:
df

,Date,Game_Type,Team,Conference,Opponent,Location,Result,Adj_Offense,Adj_Defense,Offensive_Efficiency,...,Turnover_Percent,Offensive_Rebound_Percent,Free_Throw_Rate,Opponent_Offensive_Efficiency,Opponent_Effective_FG_Percent,Opponent_Turnover_Percent,Opponent_Offensive_Rebound_Percent,Opponent_Free_Throw_Rate,Opponent_Conference,Season
0,12/23/14,0,Abilene Christian,Slnd,Arkansas Pine Bluff,N,"W, 69-61",100.2,107.6,106.8,...,27.9,25.0,35.6,94.4,42.5,21.7,34.3,37.7,SWAC,2015
1,12/23/14,0,Arkansas Pine Bluff,SWAC,Abilene Christian,N,"L, 69-61",88.0,119.1,94.4,...,21.7,34.3,37.7,106.8,64.4,27.9,25.0,35.6,Slnd,2015
2,12/20/14,0,Abilene Christian,Slnd,Boise St.,A,"L, 77-33",73.1,116.0,58.0,...,15.8,25.6,10.9,135.3,69.2,21.1,45.0,11.5,MWC,2015
3,12/20/14,0,Boise St.,MWC,Abilene Christian,H,"W, 77-33",114.4,77.2,135.3,...,21.1,45.0,11.5,58.0,25.5,15.8,25.6,10.9,Slnd,2015
4,12/28/14,0,Abilene Christian,Slnd,Grand Canyon,A,"L, 83-81",100.9,108.6,109.3,...,14.8,12.9,27.1,112.0,66.0,21.6,20.8,74.5,WAC,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10993,11/23/14,0,Youngstown St.,Horz,UNC Greensboro,N,"W, 76-67",107.4,106.0,113.8,...,19.5,32.3,54.0,100.3,50.8,19.5,25.0,16.9,SC,2015
10994,1/4/15,1,Valparaiso,Horz,Youngstown St.,H,"W, 79-64",103.8,98.1,117.6,...,7.4,50.0,33.8,95.2,44.1,13.4,26.2,35.6,Horz,2015
10995,1/4/15,1,Youngstown St.,Horz,Valparaiso,A,"L, 79-64",107.2,110.5,95.2,...,13.4,26.2,35.6,117.6,38.3,7.4,50.0,33.8,Horz,2015
10996,2/15/15,1,Wright St.,Horz,Youngstown St.,H,"L, 74-69",100.4,118.0,110.4,...,17.6,33.3,11.9,118.4,63.7,17.6,18.2,19.6,Horz,2015


col 2 is probably game type? 0 is Non-Conference, 1 is conference, 2 is conference tourney, 3 is NCAA tourney (generally? might be exceptions where 3 is other post season tourney)